### Overview

In this workbook, we'll use two movie reviews, one positive, one negative, to train a machine learning model to asses sentiment. This isn't enough data for a meaningful model, and we should not expect predictable or consistent results. However, it can be much easier to see what's happening with a very small dataset.

First, we'll create a couple of reviews. The sentiment list has two categories - 1 for positive, 0 for negative, corresponding to each review

In [ ]:
reviews = ['excellent film, excellent acting, well written screenplay, coherent plot',
    'mediocre film, unconvincing acting, stilted dialog, incoherent plot']
sentiments = [1, 0]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'review': reviews, 'sentiment': sentiments})

In [ ]:
df

### Bag-Of-Words

To create document vectors for each review, we will first need a full list of all the words used in the collection (in this case, the two documents).

We'll do this using the CountVectorizer library from scikit-learn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()

We provide *all* the rows of the reviews column 

In [ ]:
X_train = vectorizer.fit_transform(df['review'])

...to extract all terms in the entire document collection

In [ ]:
vectorizer.get_feature_names()

...and create a matrix (in sparse matrix form) showing the frequency of each term in the two documents

In [ ]:
print(X_train)

Normally, you'd want to keep term frequency matrices in sparse form (imagine a document collection with tens or hundreds of thousands of terms, where only a few show up in each document!). For our small collection, we can take a look at the full matrix.

In [ ]:
X_train.todense()

A dataframe may make it easier to visualize the relanship between the bag of words (the document vocabulary) and the document term vector for each individual record

In [ ]:
df_model = pd.DataFrame({'word': vectorizer.get_feature_names(), 
              'positive': X_train.todense().tolist()[0],
              'negative': X_train.todense().tolist()[1]})

In [ ]:
df_model

### Train an ML model

Now that we have a vector for each record, and the classification for that vector (positive or negative), we can use these records and their classification to train a ML model to determine sentiment based on term frequency.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

We'll start with a random forest classifier. We provide the document term vectors (X_train), and the classifications (sentiment scores). 

Note - once you've gotten your data into document term vectors, scikit-learn makes it easy to swap out different algorithms. 

In [ ]:
clf = RandomForestClassifier().fit(X_train, df['sentiment'])

### Feature importances

Some algorithms provide valuable analysis on your model, even if you don't use it as a predictive tool. Scikit-learn's random forest implementation will provide information on feature imporance, the impact different terms are having on the overall predictiveness of the model (see the scikit-learn documentation for more info).

Note - because our training data set is small, there will be a heavy element of randomness in the feature importances here. In fact, if you run this workbook repeatedly, you should see substantial variance in these numbers. This tends to be smoothed out with much later datasets and a larger number of trees. 

In [ ]:
clf.feature_importances_

In [ ]:
df_model['feature_importance'] = clf.feature_importances_

In [ ]:
df_model

### Make predictions

Let's create two new reviews, positive and negative, and see how our model predicts their sentiment scores.

In [ ]:
positive_review = "excellent film, acting was so so by but the plot was well thought out"

In [ ]:
negative_review = "mediocre acting, everything about this was unconvincing, save your money"

as before, we need to transform these sentences into a term frequency vector, using the bag of words we created for our training set.

In [ ]:
positive_vector = vectorizer.transform([positive_review])

In [ ]:
positive_vector.todense()

In [ ]:
negative_vector = vectorizer.transform([negative_review])

In [ ]:
negative_vector.todense()

In [ ]:
df_results = pd.DataFrame({'word': vectorizer.get_feature_names(), 
              'positive': positive_vector.todense().tolist()[0],
              'negative': negative_vector.todense().tolist()[0]})

In [ ]:
df_results

scikit-learn random forest provides two output types - a discrete category (1 or 0), or a probability for each category.

In [ ]:
clf.predict(positive_vector)

In [ ]:
clf.predict(negative_vector)

In [ ]:
clf.predict_proba(positive_vector)

In [ ]:
clf.predict_proba(negative_vector)

In [ ]:
# execise 1 - change the training and test phrases
# what effect does this have on feature importance and prediction?

# exercise 2 - try different ML classification algorithms. 
# from sklearn.neural_network import MLPClassifier
# from sklearn.naive_bayes import MultinomialNB